# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [43]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [44]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [45]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2181742537676,
 'week52high': 140.53,
 'week52low': 57.72,
 'week52change': 0.825047469139181,
 'sharesOutstanding': 17004501809,
 'float': 0,
 'avg10Volume': 89867937,
 'avg30Volume': 95710680,
 'day200MovingAvg': 108.15,
 'day50MovingAvg': 118.49,
 'employees': 0,
 'ttmEPS': 3.3,
 'ttmDividendRate': 0.8236145864404255,
 'dividendYield': 0.006810649677989012,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-29',
 'nextEarningsDate': '0',
 'peRatio': 37.450442800127135,
 'beta': 1.1606841396553729,
 'maxChangePercent': 47.40919308768932,
 'year5ChangePercent': 3.765329030127687,
 'year2ChangePercent': 2.049997011633156,
 'year1ChangePercent': 0.8435484454204317,
 'ytdChangePercent': 0.7134526190037775,
 'month6ChangePercent': 0.452696098835747,
 'month3ChangePercent': 0.09564068998091552,
 'month1ChangePercent': 0.02734467390374901,
 'day30ChangePercent': 0.027351218768559213,
 'day5ChangePercent': 0.0013202057904959936}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [46]:
data['year1ChangePercent']

0.8435484454204317

In [47]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [48]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,121.64,0.417074,N/A
1,AAL,17.52,-0.399402,N/A
2,AAP,163.29,0.0167394,N/A
3,AAPL,126.02,0.833421,N/A
4,ABBV,110.12,0.273301,N/A
...,...,...,...,...
500,YUM,107.61,0.0823538,N/A
501,ZBH,142.02,-0.0325135,N/A
502,ZBRA,381.81,0.480398,N/A
503,ZION,43.38,-0.171694,N/A


In [49]:
data

{'YUM': {'stats': {'companyName': 'Yum Brands Inc.',
   'marketcap': 32229543083,
   'week52high': 108.63,
   'week52low': 58.2,
   'week52change': 0.08161175727022901,
   'sharesOutstanding': 305052704,
   'float': 0,
   'avg10Volume': 1403407,
   'avg30Volume': 1418674,
   'day200MovingAvg': 96.71,
   'day50MovingAvg': 106.5,
   'employees': 0,
   'ttmEPS': 3.39,
   'ttmDividendRate': 1.9035160104190716,
   'dividendYield': 0.01777242464388051,
   'nextDividendDate': '0',
   'exDividendDate': '2020-11-19',
   'nextEarningsDate': '0',
   'peRatio': 31.53130962865906,
   'beta': 0.9778722930138637,
   'maxChangePercent': 13.061277846047245,
   'year5ChangePercent': 1.3198342172183664,
   'year2ChangePercent': 0.2200232573472923,
   'year1ChangePercent': 0.08235376990571945,
   'ytdChangePercent': 0.07557845513947967,
   'month6ChangePercent': 0.17565144654072232,
   'month3ChangePercent': 0.1612446044215059,
   'month1ChangePercent': 0.06243246982175612,
   'day30ChangePercent': 0.0624

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [50]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,13.51,2.19666,N/A
1,NVDA,537.38,1.3685,N/A
2,LB,39.98,1.18798,N/A
3,AMD,94.26,1.18338,N/A
4,ALB,141.77,1.09457,N/A
5,PYPL,215.67,1.06137,N/A
6,NOW,540.25,1.04138,N/A
7,ALGN,507.91,0.932916,N/A
8,FCX,25.51,0.932099,N/A
9,AAPL,126.02,0.833421,N/A


In [51]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [52]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,13.51,2.19666,1480
1,NVDA,537.38,1.3685,37
2,LB,39.98,1.18798,500
3,AMD,94.26,1.18338,212
4,ALB,141.77,1.09457,141
5,PYPL,215.67,1.06137,92
6,NOW,540.25,1.04138,37
7,ALGN,507.91,0.932916,39
8,FCX,25.51,0.932099,784
9,AAPL,126.02,0.833421,158


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [53]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [54]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,121.27,N/A,0.418029,N/A,0.3959,N/A,0.211294,N/A,0.0863993,N/A,N/A
1,AAL,17.44,N/A,-0.382681,N/A,0.0192136,N/A,0.328976,N/A,0.454624,N/A,N/A
2,AAP,164.97,N/A,0.0173303,N/A,0.166526,N/A,0.0225798,N/A,0.0303248,N/A,N/A
3,AAPL,126.64,N/A,0.819922,N/A,0.451119,N/A,0.0966751,N/A,0.0271375,N/A,N/A
4,ABBV,108.32,N/A,0.277069,N/A,0.186998,N/A,0.203618,N/A,0.0919639,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.25,N/A,0.0835386,N/A,0.178796,N/A,0.157111,N/A,0.0627549,N/A,N/A
501,ZBH,147.77,N/A,-0.0327415,N/A,0.139952,N/A,0.0196997,N/A,-0.0472098,N/A,N/A
502,ZBRA,389.02,N/A,0.479189,N/A,0.464049,N/A,0.517101,N/A,0.112784,N/A,N/A
503,ZION,43.40,N/A,-0.16974,N/A,0.219996,N/A,0.34572,N/A,0.147458,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [55]:
print(hqm_dataframe[hqm_dataframe.isnull().any(axis=1)])
hqm_dataframe.shape[0]
hqm_dataframe=hqm_dataframe.dropna()
#Dropping rowthat has none values
print(hqm_dataframe[hqm_dataframe.isnull().any(axis=1)],hqm_dataframe.shape[0])

    Ticker   Price Number of Shares to Buy One-Year Price Return  \
118    CTL  11.000                     N/A                  None   
165   ETFC  49.810                     N/A                  None   
326    MYL  16.007                     N/A                  None   
327    NBL   8.860                     N/A                  None   

    One-Year Return Percentile Six-Month Price Return  \
118                        N/A                   None   
165                        N/A                   None   
326                        N/A                   None   
327                        N/A                   None   

    Six-Month Return Percentile Three-Month Price Return  \
118                         N/A                     None   
165                         N/A                     None   
326                         N/A                     None   
327                         N/A                     None   

    Three-Month Return Percentile One-Month Price Return  \
118         

In [56]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col=f'{time_period} Price Return'
        percentile_col=f'{time_period} Return Percentile'
        #print(hqm_dataframe[change_col],hqm_dataframe.loc[row, change_col],change_col,hqm_dataframe.loc[row, percentile_col])
        #print(stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col]))
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

0       88.024
1      2.99401
2      39.9202
3      97.8044
4      77.2455
        ...   
500    50.6986
501    31.9361
502    90.8184
503    13.9721
504    79.8403
Name: One-Year Return Percentile, Length: 501, dtype: object
0      80.4391
1      12.1756
2      37.3253
3      87.0259
4      41.7166
        ...   
500    39.3214
501    33.7325
502    87.6248
503    50.0998
504    43.9122
Name: Six-Month Return Percentile, Length: 501, dtype: object
0      64.8703
1      84.8303
2      20.7585
3      39.7206
4      63.0739
        ...   
500    51.6966
501    19.9601
502    95.8084
503    87.2255
504    17.3653
Name: Three-Month Return Percentile, Length: 501, dtype: object
0      63.4731
1      98.6028
2      43.3134
3      41.1178
4      66.6667
        ...   
500    56.0878
501    7.78443
502    73.2535
503    81.8363
504    12.1756
Name: One-Month Return Percentile, Length: 501, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,121.27,N/A,0.418029,88.024,0.3959,80.4391,0.211294,64.8703,0.0863993,63.4731,N/A
1,AAL,17.44,N/A,-0.382681,2.99401,0.0192136,12.1756,0.328976,84.8303,0.454624,98.6028,N/A
2,AAP,164.97,N/A,0.0173303,39.9202,0.166526,37.3253,0.0225798,20.7585,0.0303248,43.3134,N/A
3,AAPL,126.64,N/A,0.819922,97.8044,0.451119,87.0259,0.0966751,39.7206,0.0271375,41.1178,N/A
4,ABBV,108.32,N/A,0.277069,77.2455,0.186998,41.7166,0.203618,63.0739,0.0919639,66.6667,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,109.25,N/A,0.0835386,50.6986,0.178796,39.3214,0.157111,51.6966,0.0627549,56.0878,N/A
497,ZBH,147.77,N/A,-0.0327415,31.9361,0.139952,33.7325,0.0196997,19.9601,-0.0472098,7.78443,N/A
498,ZBRA,389.02,N/A,0.479189,90.8184,0.464049,87.6248,0.517101,95.8084,0.112784,73.2535,N/A
499,ZION,43.40,N/A,-0.16974,13.9721,0.219996,50.0998,0.34572,87.2255,0.147458,81.8363,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [57]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)


In [61]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
177,FCX,25.57,N/A,0.931029,98.6028,1.35194,99.8004,0.517596,96.008,0.269909,91.6168,96.507
273,LB,39.75,N/A,1.20997,99.6008,1.54375,100,0.427538,93.4132,0.203495,88.4232,95.3593
286,LRCX,500.40,N/A,0.769874,96.8064,0.775341,97.8044,0.699856,98.4032,0.178703,85.4291,94.6108
23,ALB,144.19,N/A,1.12467,99.2016,0.829902,98.2036,0.485232,95.01,0.177515,85.2295,94.4112
29,AMAT,89.70,N/A,0.484476,91.4172,0.574301,94.012,0.635014,97.8044,0.273353,92.016,93.8124
24,ALGN,527.52,N/A,0.90465,98.4032,1.04991,99.2016,0.618441,97.6048,0.123084,76.2475,92.8643
458,URI,243.08,N/A,0.484358,91.2176,0.697704,96.008,0.393126,90.02,0.196789,87.4251,91.1677
444,TWTR,52.40,N/A,0.700792,95.8084,0.543182,92.2156,0.331444,85.4291,0.208786,88.8224,90.5689
322,MU,71.40,N/A,0.396346,86.2275,0.450442,86.8263,0.535552,96.6068,0.261336,91.2176,90.2196
491,XLNX,146.41,N/A,0.515283,93.014,0.610822,95.01,0.467291,94.8104,0.129577,77.8443,90.1697


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [58]:
hqm_dataframe.sort_values('HQM Score', ascending = False,inplace=True)
hqm_dataframe = hqm_dataframe[:51]

C:\Users\yashw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [65]:
hqm_dataframe.reset_index(drop = True, inplace = True)

In [66]:
portfolio_input()

Enter the value of your portfolio:1000000


In [68]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    #print(hqm_dataframe['Price'][i])
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,25.57,766,0.931029,98.6028,1.35194,99.8004,0.517596,96.008,0.269909,91.6168,96.507
1,LB,39.75,493,1.20997,99.6008,1.54375,100,0.427538,93.4132,0.203495,88.4232,95.3593
2,LRCX,500.40,39,0.769874,96.8064,0.775341,97.8044,0.699856,98.4032,0.178703,85.4291,94.6108
3,ALB,144.19,135,1.12467,99.2016,0.829902,98.2036,0.485232,95.01,0.177515,85.2295,94.4112
4,AMAT,89.70,218,0.484476,91.4172,0.574301,94.012,0.635014,97.8044,0.273353,92.016,93.8124
5,ALGN,527.52,37,0.90465,98.4032,1.04991,99.2016,0.618441,97.6048,0.123084,76.2475,92.8643
6,URI,243.08,80,0.484358,91.2176,0.697704,96.008,0.393126,90.02,0.196789,87.4251,91.1677
7,TWTR,52.40,374,0.700792,95.8084,0.543182,92.2156,0.331444,85.4291,0.208786,88.8224,90.5689
8,MU,71.40,274,0.396346,86.2275,0.450442,86.8263,0.535552,96.6068,0.261336,91.2176,90.2196
9,XLNX,146.41,133,0.515283,93.014,0.610822,95.01,0.467291,94.8104,0.129577,77.8443,90.1697
